In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import folium

In [3]:
train=pd.read_csv("Datasets/test.csv")
mrtExistingDf = pd.read_csv("Datasets/auxiliary-data/sg-mrt-existing-stations.csv")


In [4]:
mrtExistingDf.drop_duplicates()

,code,name,opening_year,latitude,longitude
0,NS1,Jurong East,1990,1.333295,103.742154
1,NS2,Bukit Batok,1990,1.349035,103.749526
2,NS3,Bukit Gombak,1990,1.358663,103.751913
3,NS4,Choa Chu Kang,1990,1.385363,103.744371
4,NS5,Yew Tee,1996,1.397476,103.747418
...,...,...,...,...,...
157,TE17,Outram Park,2022,1.280242,103.840052
158,TE18,Maxwell,2022,1.280533,103.843884
159,TE19,Shenton Way,2022,1.277717,103.850368
160,TE20,Marina Bay,2022,1.276311,103.854605


In [5]:
train['rent_approval_date'] = pd.to_datetime(train['rent_approval_date'], format='%Y-%M')
mrtExistingDf['opening_year'] = pd.to_datetime(mrtExistingDf['opening_year'], format='%Y')

In [6]:
train["MRT_1km"] = train["longitude"]
train["nearest_MRT_name"] = train["longitude"]
train["nearest_MRT_dis"] = train["longitude"]
from geopy.distance import geodesic
for index in tqdm(train.index):
    latitude, longitude = train.loc[index, "latitude"], train.loc[index, "longitude"]
    nearest_MRT_dis = 1000000000
    nearest_MRT_name = "None"
    MRT_1km = 0
    for ind in mrtExistingDf.index:
        if mrtExistingDf.loc[ind, 'opening_year'] > train.loc[index, 'rent_approval_date']:
            # print("date due", mrtExistingDf.loc[ind, 'opening_year'], train.loc[index, 'rent_approval_date'])
            continue
        lat2, long2 = mrtExistingDf.loc[ind, "latitude"], mrtExistingDf.loc[ind, "longitude"]
        distance = geodesic((latitude, longitude), (lat2, long2)).kilometers
        if distance < 1:
            MRT_1km += 1
        if distance < nearest_MRT_dis:
            nearest_MRT_dis = distance
            nearest_MRT_name =  mrtExistingDf.loc[ind, "name"]
    train.loc[index, "MRT_1km"] = MRT_1km
    train.loc[index, "nearest_MRT_dis"] = nearest_MRT_dis
    train.loc[index, "nearest_MRT_name"] = nearest_MRT_name
train.to_csv("Datasets/with-mrt-test.csv", index=False)

100%|██████████| 30000/30000 [17:44<00:00, 28.19it/s]


In [12]:
formal=pd.read_csv("Datasets/test.csv")
train=pd.read_csv("Datasets/with-mrt-test.csv")
train["rent_approval_date"]=formal["rent_approval_date"]

In [13]:
def transform(x):
    if x<=0.3:
        return x+0.3
    else:
        return 0.9-x

transform=np.vectorize(transform)
train['nearest_MRT_dis']=transform(train['nearest_MRT_dis'])

In [14]:
formal=pd.read_csv("Datasets/with-mrt-train.csv")
mrt_dict=dict(zip(formal["nearest_MRT_name"],formal["mrt_average_price"]))
train["mrt_average_price"]=train["nearest_MRT_name"].map(mrt_dict)

In [15]:
train.head()

,rent_approval_date,town,block,street_name,flat_type,flat_model,floor_area_sqm,furnished,lease_commence_date,latitude,longitude,elevation,subzone,planning_area,region,MRT_1km,nearest_MRT_name,nearest_MRT_dis,mrt_average_price
0,2023-01,hougang,245,hougang street 22,5-room,improved,121.0,yes,1984,1.358411,103.891722,0.0,lorong ah soo,hougang,north-east region,1.0,Kovan,0.078970,2529.932302
1,2022-09,sembawang,316,sembawang vista,4-room,model a,100.0,yes,1999,1.446343,103.820817,0.0,sembawang central,sembawang,north region,1.0,Sembawang,0.593808,2531.304665
2,2023-07,clementi,708,Clementi West Street 2,4-room,new generation,91.0,yes,1980,1.305719,103.762168,0.0,clementi west,clementi,west region,0.0,Commonwealth,-0.192259,2636.174071
3,2021-08,jurong east,351,Jurong East Street 31,3 room,model a,74.0,yes,1986,1.344832,103.730778,0.0,yuhua west,jurong east,west region,1.0,Chinese Garden,0.538309,2481.346424
4,2022-03,jurong east,305,jurong east street 32,5-room,improved,121.0,yes,1983,1.345437,103.735241,0.0,yuhua west,jurong east,west region,1.0,Chinese Garden,0.452393,2481.346424


In [ ]:
train.to_csv("Datasets/with-mrt-test.csv", index=False)